# MCMC Experiments

Let's look at what we need to get running, so that we can start to evaluate some MCMC models this semester.

In [7]:
from bsmodel import BSModel
from pathlib import Path
from randomwalk import RW

import bridgestan as bs
import cmdstanpy as csp
import matplotlib.pyplot as plt
import numpy as np
import tools as tls

import json
import pprint
import yaml

I like to put all the general config into a yaml file, and then read this in to configure the various models, parameters, and datasets necessary to evaluate an algorithm.

In [8]:
with open("config.yaml") as f:
    cfg = yaml.safe_load(f)

pprint.pp(cfg)

{'print': True,
 'digits': 4,
 'iterations': 1000,
 'warmup': 1000,
 'replications': 20,
 'seed': None,
 'stepsize': 0.24,
 'switch_limit': 3,
 'segment_length': 2,
 'db': 'experiments.db',
 'bs_path': '~/bridgestan',
 'model_path': 'stan',
 'model_name': 'normal',
 'init': {'warmup': 10000, 'iterations': 20000}}


Next, we set up the C++ code that some of these models depend on.  To get this set up on your machine, you'll need to follow the install instructions at [BridgeStan](https://roualdes.github.io/bridgestan/latest/getting-started.html) before running the code below.  Unfortunately, the install instructions for BridgeStan point you to install instructions for Stan, and this is a necessary step.

In [9]:
bs.set_bridgestan_path(Path(cfg["bs_path"]).expanduser())
stan_file, data_file = tls.get_stan_files(cfg)

In [10]:
stan_file

'stan/normal.stan'

In [12]:
bs_model = BSModel(stan_file = 'normal3d.stan',
                   data_file = data_file)

FileNotFoundError: File 'normal3d.stan' does not exist

Once all that is set up, we can finally run an algorithm on a model.  Here, we'll run the Random Walk algorithm on the model named 'normal'.

In [ ]:
rw = RW(bs_model, cfg["stepsize"], cfg["seed"])
thetas = rw.sample_constrained(cfg["iterations"])

The output of these MCMC algorithms is essentially just a collection of points, which are intelligently sampled from the distribution we're trying to characterize.  These algorithms take time to warm up, so we exclude the "warmup" set and only consider the points sampled after the warmup set.  We generally don't know when the warmup set ends, so we just make a guess and hope for the best.

In [ ]:
thetas = thetas[cfg["warmup"]:, :]
m = thetas.mean()
s = thetas.std()

Most often, we calculate the means and standard deviation of these points, taken across each dimension if there are multiple dimensions, and compare the mean(s) and standard deviation(s) to the true values.  So we can only realistically evaluate these algorithms on models for which we already know the true values.  For the "normal" model here, the mean should be $0$ and the standard deviation should be $1$.

In [ ]:
print(f"mean = {np.round(m, cfg['digits'])}")
print(f"std = {np.round(s, cfg['digits'])}")

mean = 0.0117
std = 0.9725


From this algorithm, these approximations are considered good enough.  The only real way to do better is to increase the number of iterations of the algorithm, or find a better algorithm which gets better estimates for the same number of iterations.  